In [26]:
import h5py
import numpy as np

## Initialize Parameters for L Neural Network

In [3]:
def initialize_parameters(layer_dim, seed = 3):

    """
    Arguments:
    layer_dim: Array with the dimensions of each layer of the network
    seed: random seed
    
    Return:
    W: weight matrix (n_l, n_l-1)
    b: bias vector (n_l, 1)
    
    """

    np.random.seed(seed)
    
    parameters = {}
    
    L = len(layer_dim) # Layers in the network
    
    for l in range(1, L):
        parameters["W" + str(l)] = np.random.randn(layer_dim[l], layer_dim[l-1]) * 0.01
        parameters["b" + str(l)] = np.zeros((layer_dim[l], 1))

    return parameters

**This is an example for Neural Network**


<img src="Images/1.png" width="600">   

## Forward Propagation

<img src="Images/2.png" width="600">  

<img src="Images/3.png" width="600">  

In [9]:
def linear_forward(A, W, b):

    """
    Arguments
    A: Values form the previous layer (A0 is the input layer with X values)
    W: weight matrix
    b: bias vector
    
    Return
    Z: Preactivaction parameter
    cache: a python tuple containing "A", "W" and "b" ; stored for computing the backward pass
    
    """

    Z = np.dot(W, A) + b

    cache = (A, W, b)

    return Z, cache

In [10]:
# GRADED FUNCTION: linear_activation_forward

def linear_activation_forward(A_prev, W, b, activation):
    """
    Arguments
    A_prev: activations from previous layer (or input data)
    W: weights matrix
    b: bias vector
    activation: the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A: the output of the activation function
    cache: a python tuple containing "linear_cache" and "activation_cache" stored for computing the backward pass
    """
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = (1/(1+np.exp(-Z)), Z)
        
    elif activation == "relu":                      
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = (np.maximum(0, Z), Z)
        
    cache = (linear_cache, activation_cache)

    return A, cache

In [11]:
def L_model_forward(X, parameters):
    """
    Arguments
    X: data
    parameters: output of initialize_parameters()
    
    Returns:
    AL: activation value from the output (last) layer
    caches: list of caches containing: every cache of linear_activation_forward() (there are L of them, indexed from 0 to L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    # The for loop starts at 1 because layer 0 is the input
    for l in range(1, L):
        A_prev = A 
        W = parameters[f"W{l}"]
        b = parameters[f"b{l}"]
        A, cache = linear_activation_forward(A_prev, W, b, activation="relu")
        caches.append(cache)
        
    
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    W = parameters[f"W{L}"]
    b = parameters[f"b{L}"]
    AL, cache = linear_activation_forward(A, W, b, activation="sigmoid")
    caches.append(cache)
  
          
    return AL, caches

## Cost Function

<img src="Images/4.png" width="600">  

In [14]:
def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions
    Y -- true "label" vector 

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]
    
    cost = -1/m * np.sum(Y * np.log(AL) + (1 - Y) * np.log(1 - AL))
         
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).

    return cost

## Backwward Propagation

<img src="Images/5.png" width="600">  

In [6]:
def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = (1 / m) * np.dot(dZ, A_prev.T)
    db = (1 / m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    
    
    return dA_prev, dW, db

In [17]:
def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    
    linear_cache, activation_cache = cache
    Z = activation_cache
    
    if activation == "relu":
        
        dZ = np.array(dA, copy=True)
        dZ = dZ[Z <= 0] = 0
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
  
    elif activation == "sigmoid":
        s = 1 / (1 + np.exp(-Z))
        dZ = dA * s * (1 - s)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    
    return dA_prev, dW, db

In [21]:

def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    

    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
    current_cache = caches[L - 1]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dAL, current_cache, activation="sigmoid")
    grads["dA" + str(L - 1)] = dA_prev_temp
    grads["dW" + str(L)] = dW_temp
    grads["db" + str(L)] = db_temp
    
     
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 

        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation="relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        

    return grads

In [23]:
def update_parameters(params, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    params -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    parameters = copy.deepcopy(params)
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.

    for l in range(L):
        parameters["W" + str(l + 1)] -= learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l + 1)] -= learning_rate * grads["db" + str(l + 1)]
    return parameters

In [32]:
# Load dataset
train_path = 'datasets/train_catvsnoncat.h5'
test_path = 'datasets/test_catvsnoncat.h5'
train = h5py.File(train_path, "r")
test = h5py.File(test_path, "r")

# Extract data
X_train = np.array(train["train_set_x"][:])  
y_train = np.array(train["train_set_y"][:])  
X_test = np.array(test["test_set_x"][:])     
y_test = np.array(test["test_set_y"][:]) 

In [34]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(209, 64, 64, 3) (209,)
(50, 64, 64, 3) (50,)


In [36]:
# Reshape image data to (num_samples, 64*64*3)
X_train_flat = X_train.reshape(X_train.shape[0], -1) / 255.0
X_test_flat = X_test.reshape(X_test.shape[0], -1) / 255.0

# Reshape labels to column vector (num_samples, 1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

print("X_train_flat:", X_train_flat.shape)   # Should be (209, 12288)
print("y_train:", y_train.shape)             # Should be (209, 1)

print("X_test_flat:", X_test_flat.shape)     # Should be (50, 12288)
print("y_test:", y_test.shape)               # Should be (50, 1)

X_train_flat: (209, 12288)
y_train: (209, 1)
X_test_flat: (50, 12288)
y_test: (50, 1)
